## Random Forests Are Worse At Predicting Certain Samples and Targets In Conditioned Datasets

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc

import pickle 

data_location = "../data/aging_brain/"

young = pickle.load(open(data_location + "aging_brain_young.pickle",mode='rb'))
old = pickle.load(open(data_location + "aging_brain_old.pickle",mode='rb'))

filtered = pickle.load(open(data_location + "aging_brain_filtered.pickle",mode='rb'))

batch_encoding = np.loadtxt(data_location + 'aging_batch_encoding.tsv')
batch_encoding = batch_encoding.astype(dtype=bool)

young_mask = np.zeros(37069,dtype=bool)
old_mask = np.zeros(37069,dtype=bool)

young_mask[:young.shape[0]] = True
old_mask[young.shape[0]:] = True

In [ ]:
# sc.pp.neighbors(filtered)
# sc.tl.umap(filtered)
# sc.tl.louvain(filtered)
# sc.pl.umap(filtered,color='louvain')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc

import sys
# sys.path.append('/localscratch/bbrener1/rusty_forest_v3/src')
sys.path.append('../src')
import tree_reader as tr 
import lumberjack

data_location = "../data/aging_brain/"

# forest = tr.Forest.load(data_location + 'scanpy_cmp_aging_brain_true_l1')
forest = tr.Forest.load(data_location + 'full_clustering_predicted')
forest.arguments


In [ ]:
forest.tsne_coordinates = filtered.obsm["X_umap"][young_mask]

In [ ]:
# forest.old_predictions = forest.predict(old.X)
# forest.young_predictions = forest.predict(young.X)
# forest.young_predictions.node_sample_encoding()
# forest.old_predictions.node_sample_encoding()

young_residuals = forest.young_predictions.residuals()
old_residuals = forest.old_predictions.residuals()


In [ ]:
forest.backup(data_location + "full_clustering_predicted")

In [ ]:
forest.young_predictions.prediction_report()

In [ ]:
null_young_residuals = np.power(young.X - np.mean(young.X,axis=0),2)
null_old_residuals = np.power(old.X - np.mean(old.X,axis=0),2)

In [ ]:
young_sample_absolute_residuals = np.sum(np.abs(young_residuals),axis=1)
young_sample_square_residuals = np.sum(np.power(young_residuals,2),axis=1)


In [ ]:
# plt.figure()
# plt.scatter(*filtered.obsm['X_umap'][young_mask].T,c=young_sample_absolute_residuals,s=3)
# plt.colorbar()
# plt.show()

plt.figure()
plt.scatter(*filtered.obsm['X_umap'][young_mask].T,c=young_sample_square_residuals,s=3)
plt.colorbar()
plt.show()



In [ ]:
old_sample_absolute_residuals = np.sum(np.abs(old_residuals),axis=1)
old_sample_square_residuals = np.sum(np.power(old_residuals,2),axis=1)


In [ ]:
# plt.figure()
# plt.scatter(*filtered.obsm['X_umap'][old_mask].T,c=old_sample_absolute_residuals,s=3)
# plt.colorbar()
# plt.show()

plt.figure()
plt.scatter(*filtered.obsm['X_umap'][old_mask].T,c=old_sample_square_residuals,s=3)
plt.colorbar()
plt.show()

In [ ]:
plt.figure()
plt.scatter(young_sample_absolute_residuals,young_sample_young_node_residuals,s=1)
plt.show()

plt.figure()
plt.scatter(old_sample_absolute_residuals,old_sample_young_node_residuals,s=1)

In [ ]:
plt.figure()
plt.scatter(node_young_mean_residuals,node_old_mean_residuals,s=1)
plt.show()

In [ ]:
# young_nsr2 = forest.young_predictions.node_sample_r2()
# forest.old_predictions.nsr2 = None
# old_nsr2 = forest.old_predictions.node_sample_r2()

# young_node_size = np.sum(forest.young_predictions.node_sample_encoding(),axis=1)
# old_node_size = np.sum(forest.old_predictions.node_sample_encoding(),axis=1)

# young_node_mean_r2 = np.sum(young_nsr2,axis=1) / young_node_size
# old_node_mean_r2 = np.sum(old_nsr2,axis=1) / old_node_size

# plt.figure()
# plt.scatter(young_node_mean_r2[leaf_mask],old_node_mean_r2[leaf_mask],s=1)
# plt.show()

# old_node_mean_r2[leaf_mask]



In [ ]:
# young_sample_young_node_residuals = []    
# old_sample_young_node_residuals = []

# for i,old_sample_encoding in enumerate(forest.old_predictions.node_sample_encoding()[leaf_mask].T):
#     if i % 100 == 0:
#         print(i)
#     mean_young_node_residuals = np.mean(young_node_mean_r2[leaf_mask][old_sample_encoding])
#     mean_old_node_residuals = np.mean(old_node_mean_r2[leaf_mask][old_sample_encoding])
#     young_sample_young_node_residuals.append(mean_young_node_residuals)
#     old_sample_young_node_residuals.append(mean_old_node_residuals)


# plt.figure()
# plt.scatter(young_sample_young_node_residuals,old_sample_young_node_residuals,s=1)
# plt.plot([0,700],[0,700],color='red')
# plt.show()

# plt.figure()
# plt.scatter(young_sample_absolute_residuals,young_sample_young_node_residuals,s=1)
# plt.show()

# plt.figure()
# plt.scatter(old_sample_absolute_residuals,old_sample_young_node_residuals,s=1)

In [ ]:
node_residual_delta = np.array(old_sample_young_node_residuals) - np.array(young_sample_young_node_residuals)

plt.figure()
plt.title("Largest Changes In Node Residuals")
plt.scatter(*filtered.obsm["X_umap"][old_mask].T,cmap='viridis',c=node_residual_delta,s=1)
plt.colorbar(label="Increase in Node R2")
plt.show()

Of course we can't make entirely direct comparisons on the prediction error for samples, because we can never be certain to what degree the sample matching procedure would bias our result. (In a fundamental sense, when we attempt to match samples to each other, we will do so on the basis of their residuals relative to whatever model or error function we pick)

But we can make meaningful comparisons of the degree of mismatch for both features and node clusters:

In [ ]:
young_feature_fvu = forest.young_predicitons.feature_remaining_error(mode='mean')

In [ ]:
# First let's get a sense for the kinds of FVU values we can observe 

plt.figure()
plt.title("Null Feature FVUs (Lower Is Better)")
plt.hist(young_feature_fvu,bins=100)
plt.xlabel("FVU")
plt.ylabel("Frequency")
plt.show()



In [ ]:
# Now let's examine the quality of predictions for features in the old 
# Cells

old_feature_fvu = forest.old_predictions.feature_remaining_error(mode='mean')

# Note something interesting:

print(np.sum(old_feature_fvu > 1.))

In [ ]:

masked_fvu = old_feature_fvu.copy()
masked_fvu[old_feature_fvu > 1] = 1

plt.figure()
plt.title("Predicted Feature FVUs (Lower Is Better)")
plt.hist(masked_fvu,bins=100)
plt.xlabel("FVU")
plt.ylabel("Frequency")
plt.show()

Now we see one of the difficulties in using Fraction of Variance Left Unexplained as a comparison metric. When a new dataset is introduced to a regressor, the value can suddenly fall outside the 0-1 range. Even more troublesome: for certain genes, notably ones not expressed in the young cells, we can even experience divisions by zero. As a correlary, the Coefficient of Determination can even become negative, and thus is also challenging to use.

But of course, we are also interested in the features for which the FVU changes most substantially. After all, there are plenty of poorly predictable features even in our original dataset. 

In [ ]:
delta_fvu = old_feature_fvu - young_feature_fvu

delta_fvu_sort = np.argsort(np.abs(delta_fvu))

plt.figure()
plt.hist(delta_fvu[delta_fvu < 1],bins=100)
plt.show()

# Let's take a look at the top 20 ranked features by the difference in
# prediction quality 

print(forest.output_features[delta_fvu_sort[-20:]])
print(delta_fvu[delta_fvu_sort[-20:]])


In [ ]:
# At face value we may immediately observe that Gh (growth hormone) is an
# extremely classic gene affected by aging, as is Ave (Vassopressin), 
# Chorionic Gonadotropin (monomer of FSH,LH, and TSH to boot),

# More interestingly, their behavior is easily detectable as different 
# through the use of a random forest, but for example Lyve1 and Clec4d 
# appear to have completely insignificant differential expression, and Avp is quite 
# low-ranked. 

In [ ]:
gene = "Klk6"
gene_index = forest.truth_dictionary.feature_dictionary[gene]

# plt.figure()
# plt.hist(young.X[:,gene_index],bins=100,log=True)
# plt.show()

# plt.figure()
# plt.hist(old.X[:,gene_index],bins=100,log=True)
# plt.show()

plt.figure()
plt.scatter(*filtered.obsm['X_umap'][young_mask].T,c=young.X[:,gene_index],s=3)
plt.show()
plt.figure()
plt.scatter(*filtered.obsm['X_umap'][old_mask].T,c=old.X[:,gene_index],s=3)
plt.show()


## Factor Discrepancies 

In [ ]:

comparisons = forest.young_predictions.compare_factors(forest.old_predictions,bins=100)

# print(comparisons)

In [ ]:
clust_intr = 27
clust = forest.split_clusters[clust_intr]

# Let's sort features by young COD and Mean Delta

young_error,young_error_parent = forest.young_predictions.factor_feature_error(clust)
old_error,old_error_parent = forest.old_predictions.factor_feature_error(clust)

young_cod = 1 - (young_error+1)/(young_error_parent+1)
old_cod = 1 - (old_error+1)/(old_error_parent+1)

mean_delta = np.mean(forest.node_representation(clust.nodes,mode='additive_mean'),axis=0)

delta_sort = np.argsort(mean_delta)
cod_sort = np.argsort(young_cod)

print(young_cod[cod_sort[:20]])
print(young_cod[cod_sort[-20:]])
print(young_cod[delta_sort[20:]])
print(young_cod[delta_sort[-20:]])

# Now let's select the cells of interest to us (predicted to be common in cluster of interest)

young_scores = forest.young_predictions.factor_matrix()[:,clust_intr]
old_scores = forest.old_predictions.factor_matrix()[:,clust_intr]

plt.figure()
plt.hist(young_scores,bins=40,log=True)
plt.show()

plt.figure()
plt.hist(old_scores,bins=40,log=True)
plt.show()

# Visually ID a cutoff 

# young_factor_mask = np.abs(young_scores) > .025
# old_factor_mask = np.abs(old_scores) > .025
young_factor_mask = young_scores > .025
old_factor_mask = old_scores > .025


# Select the cells from each dataset and plot an agglomerated heatmap

young_factor_sort = np.argsort(young_scores[young_factor_mask])
old_factor_sort = np.argsort(old_scores[old_factor_mask])

young_selection = young.X[young_factor_mask][young_factor_sort[::-1]]
old_selection = old.X[old_factor_mask][old_factor_sort[::-1]]

# relevant_features = cod_sort[-200:]
relevant_features = delta_sort[-200:]

from scipy.cluster.hierarchy import linkage,dendrogram

feature_agg = dendrogram(linkage(young_selection.T[relevant_features], method='average'), no_plot=True)['leaves']

relevant_young = young_selection.T[relevant_features][feature_agg].T
relevant_old = old_selection.T[relevant_features][feature_agg].T

plt.figure(figsize=(4,3))
plt.title(f"Top 200 Most Determined Genes, Cluster {clust.name()}\n Young Cells")
plt.imshow(relevant_young,aspect='auto',interpolation='none')
plt.xlabel("Genes")
plt.ylabel("Cells")
plt.colorbar(label="Expression (Log TPM)")
plt.show()

plt.figure(figsize=(4,3))
plt.title(f"Top 200 Most Determined Genes, Cluster {clust.name()}\n Old Cells")
plt.imshow(relevant_old,aspect='auto',interpolation='none')
plt.xlabel("Genes")
plt.ylabel("Cells")
plt.colorbar(label="Expression (Log TPM)")
plt.show()

young_sample_agg = dendrogram(linkage(young_selection, method='average'), no_plot=True)['leaves']
old_sample_agg = dendrogram(linkage(old_selection, method='average'), no_plot=True)['leaves']


plt.figure(figsize=(4,3))
plt.title(f"Top 200 Most Determined Genes, Cluster {clust.name()}\n Young Cells")
plt.imshow(young_selection[young_sample_agg].T[relevant_features][feature_agg].T,aspect='auto',interpolation='none')
plt.xlabel("Genes")
plt.ylabel("Cells")
plt.colorbar(label="Expression (Log TPM)")
plt.show()

plt.figure(figsize=(4,3))
plt.title(f"Top 200 Most Determined Genes, Cluster {clust.name()}\n Young Cells")
plt.imshow(old_selection[old_sample_agg].T[relevant_features][feature_agg].T,aspect='auto',interpolation='none')
plt.xlabel("Genes")
plt.ylabel("Cells")
plt.colorbar(label="Expression (Log TPM)")
plt.show()


In [ ]:
# forest.young_predicitons.nse[0].shape
# selection = forest.young_predicitons.matrix[forest.young_predicitons.nse[0]]
# forest.young_predicitons.nsr2 = None
# forest.old_predictions.nsr2 = None
# np.sum(forest.young_predicitons.node_sample_r2(),axis=1)[12]
# young_fvus = []
# old_fvus = []
# for factor in forest.split_clusters:
#     y_f,o_f = forest.young_predictions.compare_factor_fvu(forest.old_predictions,factor)
#     young_fvus.append(y_f)
#     old_fvus.append(o_f)
fractions = []
for factor in forest.split_clusters[1:]:
    young_fractions = np.mean([forest.young_predictions.node_fraction(n) for n in factor.nodes])
    old_fractions = np.mean([forest.old_predictions.node_fraction(n) for n in factor.nodes])
    fractions.append((young_fractions,old_fractions))
    
print(list(enumerate(fractions)))
    
    # for factor in forest.split_clusters[1:]:
#     fractions.append(forest.young_predictions.compare_factor_fractions(forest.old_predictions,factor))


In [ ]:
# young_error = forest.young_predictions.factor_feature_error(forest.split_clusters[16])
# old_error = forest.old_predictions.factor_feature_error(forest.split_clusters[16])

young_cod = 1 - ((young_error[0]+1)/(young_error[1]+1))
old_cod = 1 - ((old_error[0]+1)/(old_error[1]+1))

delta = young_cod - old_cod

delta_sort = np.argsort(delta)
young_sort = np.argsort(young_cod)

print(forest.output_features[young_sort[-20:]])
print(young_cod[young_sort[-20:]])
print(old_cod[young_sort[-20:]])
# print(forest.output_features[delta_sort[-20:]])
# print(young_cod[delta_sort[-20:]])
# print(old_cod[delta_sort[-20:]])

In [ ]:
young_error = forest.young_predictions.factor_feature_error(forest.split_clusters[27])
old_error = forest.old_predictions.factor_feature_error(forest.split_clusters[27])

young_cod = 1 - ((young_error[0]+1)/(young_error[1]+1))
old_cod = 1 - ((old_error[0]+1)/(old_error[1]+1))

delta = young_cod - old_cod

delta_sort = np.argsort(delta)
young_sort = np.argsort(young_cod)

print(forest.output_features[young_sort[-20:]])
print(young_cod[young_sort[-20:]])
print(old_cod[young_sort[-20:]])
# print(forest.output_features[delta_sort[-20:]])
# print(young_cod[delta_sort[-20:]])
# print(old_cod[delta_sort[-20:]])

In [ ]:
young_error = forest.young_predictions.factor_feature_error(forest.split_clusters[47])
old_error = forest.old_predictions.factor_feature_error(forest.split_clusters[47])

young_cod = 1 - ((young_error[0]+1)/(young_error[1]+1))
old_cod = 1 - ((old_error[0]+1)/(old_error[1]+1))

delta = young_cod - old_cod

delta_sort = np.argsort(delta)
young_sort = np.argsort(young_cod)

print(forest.output_features[young_sort[-20:]])
print(young_cod[young_sort[-20:]])
print(old_cod[young_sort[-20:]])
# print(forest.output_features[delta_sort[-20:]])
# print(young_cod[delta_sort[-20:]])
# print(old_cod[delta_sort[-20:]])

In [ ]:
# young_cods = np.ones(len(young_fvus)) - young_fvus
# old_cods = np.ones(len(old_fvus)) - old_fvus
# delta = young_cods - old_cods
# ratio = delta/young_cods

print(list(enumerate(zip(young_cods,old_cods))))
print(list(enumerate(ratio)))
# plt.figure()
# plt.scatter(young_cods,old_cods)
# plt.plot([0,.35],[0,.35])
# # plt.scatter(np.arange(len(delta)),delta)
# plt.show()

In [ ]:

# plt.figure()
# plt.title(f'Umap Embedding, Mouse Brain')
# plt.scatter(*young.obsm['X_umap'].T,c=forest.sample_labels,cmap='rainbow',s=1,alpha=1)
# plt.show()

# plt.figure()
# plt.title(f'Repredicted Embedding, Young Mouse Brain')
# plt.scatter(*young_repredicted[:,-2:].T,s=1,alpha=1)
# plt.show()


# plt.figure()
# plt.title(f'Repredicted Embedding, Aged Mouse Brain')
# plt.scatter(*old_repredicted[:,-2:].T,s=1,alpha=1)
# plt.show()


# feature = "Retnlg"
# f_i = forest.truth_dictionary.feature_dictionary[feature]
# plt.figure()
# plt.title(f'{feature}')
# plt.scatter(*filtered.obsm['X_umap'][young_mask].T,c=filtered.X[:,f_i][young_mask],s=3,alpha=.4)
# plt.show()

# plt.figure()
# plt.title(f'{feature}')
# plt.scatter(*filtered.obsm['X_umap'][old_mask].T,c=filtered.X[:,f_i][old_mask],s=3,alpha=.4)
# plt.show()

# print(young_cod[f_i])
# print(old_cod[f_i])

# feature = "Fyn"
# f_i = forest.truth_dictionary.feature_dictionary[feature]

# plt.figure()
# plt.title(f'Repredicted Embedding, Young Mouse Brain')
# plt.scatter(*young_repredicted[:,-2:].T,c=young.X[:,f_i],s=1,alpha=1)
# plt.show()


# plt.figure()
# plt.title(f'Repredicted Embedding, Aged Mouse Brain')
# plt.scatter(*old_repredicted[:,-2:].T,c=old.X[:,f_i],s=1,alpha=1)
# plt.show()


feature = "Tmem119"
f_i = forest.truth_dictionary.feature_dictionary[feature]
plt.figure()
plt.title(f'{feature}')
plt.scatter(*forest.tsne_coordinates.T,c=forest.output[:,f_i],s=3,alpha=.4)
plt.colorbar()
plt.show()

feature = "Cd74"
f_i = forest.truth_dictionary.feature_dictionary[feature]
plt.figure()
plt.title(f'{feature}')
plt.scatter(*forest.tsne_coordinates.T,c=forest.output[:,f_i],s=3,alpha=.4)
plt.colorbar()
plt.show()


# Immune cells: CD45/C1qc 
# Oligodendrocytes: Pdgfra/Sox10
# Astrocytes: Aldoc/Cldn10
# mature neurons/neuroendocrine: Syt1/Snap25
# Ependymal cells: Rarres2/Dynlrb2
# Vascular cells: Cldn5/Esam

In [ ]:
# Predicting a tsne
# forest.add_output_feature(forest.tsne_coordinates.T[0],"tsne_0")
# forest.add_output_feature(forest.tsne_coordinates.T[1],"tsne_1")

# forest.output_features.shape
# forest.young_predictions.nme = None
# forest.young_predictions.node_mean_encoding()

# forest.old_predictions.nme = None
# forest.old_predictions.node_mean_encoding()


# young_repredicted = forest.young_predictions.prediction(mode="mean")
# old_repredicted = forest.old_predictions.prediction(mode="mean")


In [ ]:
# distance = np.sqrt(np.sum(np.power(forest.tsne_coordinates - young_repredicted[:,-2:],2),axis=1))

# plt.figure()
# plt.hist(distance,bins=100)
# plt.show()

# plt.figure()
# plt.scatter(forest.tsne_coordinates.T[0],young_repredicted[:,-2])
# plt.show()

# plt.figure()
# plt.scatter(forest.tsne_coordinates.T[1],young_repredicted[:,-1])
# plt.show()

# np.sum((distance > 2).astype(dtype=int))

# plt.figure()
# plt.hist(forest.tsne_coordinates.T[0] - young_repredicted[:,-2],bins=100,log=True)
# plt.show()

# plt.figure()
# plt.hist(forest.tsne_coordinates.T[1] - young_repredicted[:,-1],bins=100,log=True)
# plt.show()

In [ ]:
plt.figure()
plt.title("Changes in Factor Prediction Quality")
plt.scatter(comparisons["FVU Deltas"],-np.log10(comparisons["P values"]))
plt.xlabel("Magnitude (Change In FVU)")
plt.ylabel("Significance, Change in MSE, -Log10(P Value)")
plt.show()

In [ ]:
forest.tsne_coordinates = filtered.obsm['X_umap'][young_mask]

In [ ]:
# We need to look at the tree of these factors to really get a sense for what's going on
# forest.likely_tree

forest.html_tree_summary(n=10)

In [ ]:
filtered.obsm['X_umap']

In [ ]:
sizes = np.array([len(n.samples()) for n in forest.split_clusters[17].nodes])
sister_sizes = np.array([len(n.sister().samples()) for n in forest.split_clusters[17].nodes])

In [ ]:
plt.figure()
plt.hist(sizes,bins=100)
plt.show()

plt.figure()
plt.hist(sister_sizes,bins=100)
plt.show()

plt.figure()
plt.hist(sizes-sister_sizes,bins=100)
plt.show()

plt.figure()
plt.hist(sizes/sister_sizes,bins=100)
plt.show()

plt.figure()
plt.hist(np.log(sizes/sister_sizes),bins=100)
plt.show()


In [ ]:
for factor in reversed(forest.split_clusters[1:]): 
    print(factor.name())
    young_self,young_parent = forest.young_predicitons.factor_total_error(factor)
    young_fvu = np.sum(young_self)/ np.sum(young_parent)
    print(f"Young FVU: {young_fvu}")
    print(f"Young COD: {1-young_fvu}")
    old_self,old_parent = forest.old_predictions.factor_total_error(factor)
    old_fvu = np.sum(old_self)  / np.sum(old_parent)
    print(f"Old FVU: {old_fvu}")
    print(f"Old COD: {1-old_fvu}")b


In [ ]:
self_error,parent_error = forest.split_clusters[17].raw_error()

In [ ]:
for factor in reversed(forest.split_clusters[1:]): 
    forest.young_predictions.compare_factor_means(forest.old_predictions,factor,plot=['distance','mean','scatter','rank'])
    
    
    

In [ ]:
from scipy.stats import mannwhitneyu

# young_17_error = forest.young_predicitons.factor_total_error(forest.split_clusters[17])
# old_17_error = forest.old_predictions.factor_total_error(forest.split_clusters[17])

# young_mse = np.sum(young_17_error[0]) / (np.sum([len(n.samples()) for n in forest.split_clusters[17].nodes]) * 2000)
# old_mse = np.sum(old_17_error[0]) / (np.sum([len(n.samples()) for n in forest.split_clusters[17].nodes]) * 2000)

# within_error = np.sum(young_17_error[0]) + np.sum(old_17_error[0])

In [ ]:
print(young_mse)
print(old_mse)

In [ ]:
ratio = self_error / parent_error

In [ ]:
plt.figure()
plt.hist(self_error,log=True)
plt.show()

In [ ]:
trick_sorted,trick_ratio = forest.split_clusters[17].error_ratio()

plt.figure()
plt.hist(trick_ratio,bins=100,log=True)
plt.show()

In [ ]:
forest.young_predicitons

In [ ]:
forest.young

In [ ]:
factor = forest.split_clusters[16]

young_self, young_parent = forest.young_predicitons.factor_total_error(factor)

In [ ]:
old_self,old_parent = forest.old_predictions.factor_total_error(factor)

In [ ]:
young_fvu = (young_self+1)/(young_parent+1)
young_fvu[~np.isfinite(young_fvu)] = 0
young_sort = np.argsort(young_fvu)

In [ ]:
best_young = forest.output_features[young_sort[:10]]
best_young

In [ ]:
old_fvu = (old_self+1)/(old_parent+1)


In [ ]:
print(young_fvu[young_sort[:10]])

In [ ]:
print(old_fvu[young_sort[:10]])

In [ ]:
top_indices = [n.filter.reduction.features[0] for n in forest.split_clusters[27].nodes]

In [ ]:
mean_sort = np.argsort(np.mean(young.X,axis=0))

In [ ]:
from tree_reader_utils import jackknife_variance
from scipy.stats import sem

In [ ]:
def jackknife_variance(values):
    squared_values = np.power(values, 2)
    n = squared_values.shape[0]
    sum = np.sum(squared_values, axis=0)
    excluded_sum = sum - squared_values
    excluded_mse = excluded_sum / (n - 1)
    jackknifed = np.var(
        excluded_mse, axis=0) * (n - 1)

    return jackknifed


In [ ]:
close = np.loadtxt('/Users/bbrener1/time_series/close.txt')
log_close = np.log2(close)
log_diff = log_close[1:] - log_close[:-1]


normal = np.random.normal(size=100)

In [ ]:
sem(log_diff)

In [ ]:
jackknife_variance(normal)

In [ ]:
normal

In [ ]:
factor_matrix = forest.factor_matrix()

In [ ]:
from scipy.spatial.distance import cdist
correlation_45 = cdist(np.array([factor_matrix.T[45],]),young.X.T,metric='correlation')[0] - 1

In [ ]:
correlation_sort = np.argsort(correlation_45)

In [ ]:
correlation_45[correlation_sort]

In [ ]:
plt.figure()
plt.hist(correlation_45,bins=20)
plt.show()

In [ ]:
forest.output_features[correlation_sort[:20]]

In [ ]:
plt.figure()
plt.scatter(*forest.tsne_coordinates.T,c=factor_matrix.T[45])
plt.colorbar()
plt.show()

In [ ]:
forest.young_predictions.observed_marginal(forest.nodes())